header-item header-item--navigation

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys


import time
import pandas as pd
from urllib.parse import quote
from collections import deque


In [2]:
base_url = "https://www.lamaretail.com/"
# search_query = "Software Engineer"
# search_url = f"https://www.rozee.pk/job/jsearch/q/{quote(search_query)}"

# print(search_url)


In [ ]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url=base_url)


categories = []


try: #getting categories
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".site-nav__item.site-nav__expanded-item"))
    )

    categories_elements = driver.find_elements(By.CSS_SELECTOR, ".site-nav__item.site-nav__expanded-item")


    for element in categories_elements:
        category_name = element.get_attribute('innerText')
        if category_name is None or category_name == '':
            continue

        try:    
            category_data_link = base_url + element.find_element(By.TAG_NAME, 'summary').get_attribute('data-link')
        except Exception as e:
            category_data_link = element.find_element(By.TAG_NAME, 'a').get_attribute('href')



        print(category_name)    
        print(category_data_link)
        categories.append({"name": category_name, "link": category_data_link})


        
    
except Exception as e:
    print(f'excpetion in getting categories: {e}')
print('ok')

# driver.close()

Title = []
OriginalPrice = []
SalePrice = []
Discount = []
Link = []
Category = []
Available = []
Brand = []
Sku = []



for category in categories:
    
    try:
        driver.get(category['link'])
    except Exception as e:
        print(f'error in {category}')
    

    try: #get all products
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "grid-product__content"))
        )

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        product_set = set(driver.find_elements(By.CLASS_NAME, 'grid-product__content'))
        product_elements = deque(product_set)
        print(f'product_elements initially: {len(product_set)}')
        i = 0


        main_window = driver.current_window_handle


    except Exception as e:
        print(f'error in getting product {e}')


    main_window = driver.current_window_handle

    while len(product_elements) > 0:
        product_element = product_elements.popleft()

        
        print(f'{i}, product_elements) {len(product_elements)}')
        i += 1


        try: #dynamic scroll
            
            if len(product_elements) <= 2:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "grid-product__content"))
                )


                new_product_set = set(driver.find_elements(By.CLASS_NAME, 'grid-product__content'))
                new_product_set -= product_set
                
                product_set = product_set.union(new_product_set)
                product_elements += deque(new_product_set)

                print(f'new product set: {len(new_product_set)}')
                print(f'product_elements total: {len(product_elements)}')
        except Exception as e:
            print(f'exception in dynamic scroll: {e}')


        # product link
        try:
            link = product_element.find_element(By.CLASS_NAME, "grid-product__link").get_attribute('href')
            print(link)

        except Exception   as e:
            print(f'error in product link {e}')
            continue


        #moving to video page
        try:
            print(f'before moving {driver.current_url}')
            driver.execute_script(f"window.open('{link}', '_blank')")
            driver.switch_to.window(driver.window_handles[-1])
            print(f'after moving {driver.current_url}')
            WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "product-single__meta"))
                )
        except Exception as e:
            print(f'exception in moving to product page: {e}')


        #title
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "product-single__title"))
            )
            print('title located')
            title = driver.find_element(By.CLASS_NAME,'product-single__title').get_attribute('innerText')
            print(f'title {title}')
            sku = driver.find_element(By.CLASS_NAME,  'product-single__sku').text
            print(f'sku {sku}')

        except Exception as e:
            print(f'error in title {e}')


        #price
        try:
            try:
                original_price = driver.find_element( By.CSS_SELECTOR, ".product__price.product__price--compare").find_element(By.TAG_NAME, 'span').text
                sale_price = driver.find_element( By.CSS_SELECTOR, ".product__price.on-sale").find_element(By.TAG_NAME, 'span').text
                discount =   driver.find_element( By.CSS_SELECTOR, ".product__price-savings").text



            except Exception as e:
                print(f'not on sale')
                original_price = driver.find_element( By.CSS_SELECTOR, ".product__price").find_element(By.TAG_NAME, 'span').text
                sale_price = original_price
                discount = None

        except Exception as e:
            print(f'error in price {e}')
            continue

        
        print(original_price)
        print(sale_price)
        print(discount)



        available = None
        #availability
        try:
            try:
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".shopify-payment-button__button.shopify-payment-button__button--unbranded"))
                )
                available = True
            except:
                available = False


        except Exception as e:
            print(f'error in availabilty check {e}')
            continue
        print(available)

        brand = "lama-retail" 


        Title.append(title)
        Brand.append(brand)
        Available.append(available)
        OriginalPrice.append(original_price)
        SalePrice.append(sale_price)
        Discount.append(discount)
        Sku.append(sku)
        Link.append(link)
        Category.append(category['name'])










        driver.close()
        driver.switch_to.window(main_window)




            

driver.quit()







WOMAN
https://www.lamaretail.com//collections/woman
MAN
https://www.lamaretail.com//collections/man
SHOES
https://www.lamaretail.com//collections/shoes
ACCESSORIES
https://www.lamaretail.com//collections/accessories
SELFCARE
https://lamaretail.com/collections/self-care
FRAGRANCE
https://lamaretail.com/collections/fragrance
BANANA ROAD
PRINT SHOP
https://lamaretail.com/collections/banana-road
CLEARANCE
https://www.lamaretail.com//collections/last-chance
ok
product_elements initially: 27
0, product_elements) 26
https://lamaretail.com/collections/true-body/products/curve-muslin-shirt-was24tp090-black
before moving https://lamaretail.com/collections/woman
after moving https://lamaretail.com/collections/true-body/products/curve-muslin-shirt-was24tp090-black
title located
title CURVE MUSLIN SHIRT
sku WAS24TP090-T0X-BLK
not on sale
Rs.7,950.00
Rs.7,950.00
None
True
1, product_elements) 25
https://lamaretail.com/collections/woman-shoes-new-in/products/woman-heels-wfs25hl004-brown
before moving

In [16]:
print(len(Title))

data = {
    'Title': Title,
    'Brand': Brand,
    'Available': Available,
    'OriginalPrice': OriginalPrice,
    'SalePrice': SalePrice,
    'Discount': Discount,
    'Sku': Sku,
    'Link': Link,
    'Category': Category  # category['name'] already appended in your list
}

# Create the DataFrame
df = pd.DataFrame(data)

df.to_csv('lama-retail.csv')




202


In [17]:
# Remove "Rs." and commas from SalePrice and convert to float
df['SalePrice'] = df['SalePrice'].str.replace('Rs.', '', regex=False)\
                                 .str.replace(',', '', regex=False)\
                                 .astype(float)

# Remove "Rs." and commas from OriginalPrice and convert to float
df['OriginalPrice'] = df['OriginalPrice'].str.replace('Rs.', '', regex=False)\
                                         .str.replace(',', '', regex=False)\
                                         .astype(float)

# Verify the conversions
print(type(df["SalePrice"][0]))
print(df["SalePrice"].head())

print(type(df["OriginalPrice"][0]))
print(df["OriginalPrice"].head())


<class 'numpy.float64'>
0    7950.0
1    7450.0
2    3950.0
3    7450.0
4    7950.0
Name: SalePrice, dtype: float64
<class 'numpy.float64'>
0    7950.0
1    7450.0
2    3950.0
3    7450.0
4    7950.0
Name: OriginalPrice, dtype: float64


In [18]:



avg_price_per_category = df.groupby('Category')['SalePrice'].mean().reset_index()

print("Average Sale Price per Category:")
print(avg_price_per_category)


def most_common(series):
    return series.value_counts().idxmax()

most_common_brand = df.groupby('Category')['Brand'].agg(most_common).reset_index()

print("\nMost Common Brand per Category:")
print(most_common_brand)

Average Sale Price per Category:
                  Category    SalePrice
0              ACCESSORIES  6191.333333
1  BANANA ROAD\nPRINT SHOP  3704.791667
2                CLEARANCE  3264.268293
3                FRAGRANCE  7450.000000
4                      MAN  6465.000000
5                    SHOES  8033.333333
6                    WOMAN  6653.703704

Most Common Brand per Category:
                  Category        Brand
0              ACCESSORIES  lama-retail
1  BANANA ROAD\nPRINT SHOP  lama-retail
2                CLEARANCE  lama-retail
3                FRAGRANCE  lama-retail
4                      MAN  lama-retail
5                    SHOES  lama-retail
6                    WOMAN  lama-retail
